# Description of the problem and solution

The task1 was to predict a person's age from the brain image data: a standard regression problem. The original dataset included 832 features as well as a lot of NaN values and a few outliers. A good preprocessing stage was necessary in order to have a well defined dataset that could be used in our regression model. First step was the imputation of the dataset. Filling each NaN value with the median of each feature column. The use of the median instead of other value (e.g. mean) is justified since a lot of outliers are included in the dataset. (e.g. 1 2 _ 5 20 median: 3 mean: 7). Next step was the feature extraction. By using the "autofeat" library (paper: https://arxiv.org/pdf/1901.07329.pdf), we extracted the 21 most important features. The way the algorithm works is going through a loop of correlation of features with target, select promising features, train Lasso regression model with promising features, filter the good features keeping the ones with non-zero regression weights. We updated the datasets by keeping only the 21 most important features. Finally, we used these updated datasets for the training of our final regression model. A lot of outlier detection techniques were used but we decided to keep the outliers and use a tree-based method for our final model. Tree-methods have been proved to be robust to outliers and we avoid risking excluded important features / points from the dataset. The "ExtraTreesRegressor" model from the "sklearn" package was used and fine tuned based on the R2 score performance in our validation set. The final model had a score >0.6 in the validation sets using cross-validation and in the submission leaderboard of ETH scored 0.6812 while the hard baseline was set to 0.65 by the Advanced Machine Learning Task1 team.

# Include all the necessary packages

In [0]:
!pip install autofeat

from sklearn.metrics import r2_score

try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from autofeat import FeatureSelector

from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor

# Load the data from the CSV files

In [0]:
column_names_x = ['id']
for i in range(832):
  column_names_x.append('x'+str(i))

raw_dataset_x = pd.read_csv('/content/X_train.csv', names=column_names_x,
                      na_values = "?", comment='\t',
                      sep=",", skipinitialspace=True, skiprows=True)

dataset_x = raw_dataset_x.copy()
dataset_x.tail()

,id,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,...,x792,x793,x794,x795,x796,x797,x798,x799,x800,x801,x802,x803,x804,x805,x806,x807,x808,x809,x810,x811,x812,x813,x814,x815,x816,x817,x818,x819,x820,x821,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
1207,1207.0,NaN,5395.719279,95668.548818,1125.414599,10341.757613,10.547165,108249.187400,1.073291e+06,108672.758838,2.365013,1.042126e+06,21031.538201,1059.704248,103860.409465,107.805931,100.350289,14040.029112,NaN,90414.095308,10.108470,3.215609,102144.219482,10.087478,3825.046407,89.538601,4550.239929,3542.470382,206892.186644,1.011027e+06,11432.036864,10.957698,976.093967,84.689946,2500.095879,1068.544855,8.868114,3.695648,10381.626637,8.641068e+05,...,6.934015,2.05604,9.362337,908.498493,94.234452,10.319140,4155.345879,8.498432e+16,10977.013897,93.766125,8.120353,10595.018829,9.594486e+05,10.485803,10.601033,101328.492269,2.417968,48.359296,9486.144826,2.185578,966.094287,9.039254e+05,10.510759,1049.753163,10856.049561,9109.098919,10320.748617,10892.743222,106528.008864,294.801642,10.519831,10.234396,987.456317,978.661701,109520.061346,102914.439553,8144.701025,10.442410,102380.867791,2.236101
1208,1208.0,93669.580198,3564.454295,96937.341346,1040.828378,8415.112792,10.721800,100628.318687,1.027671e+06,102269.472299,2.398768,1.015631e+06,10073.888339,1005.671797,106924.048275,98.512805,104.366843,11719.094168,115055.546469,95750.178334,NaN,3.446025,100455.697718,10.303138,3502.012112,84.167435,3462.164372,3411.771480,206892.234016,1.082826e+06,9957.026155,11.155088,1127.806594,107.745872,2298.057201,1031.637383,8.778034,3.477553,10627.490011,1.007256e+06,...,10.732723,2.14034,10.930583,956.306694,117.419062,10.361670,3436.194754,6.249342e+16,9866.685225,104.420975,9.171168,10762.025617,9.205254e+05,10.627956,10.101622,105133.566646,2.740250,55.449159,10959.655054,2.160671,1111.015635,1.118527e+06,10.342601,1030.540259,10097.006670,9039.074820,10895.768148,10315.088493,106898.299599,195.245438,10.618901,10.456550,1112.699713,NaN,NaN,106685.647900,7428.338174,10.405107,107051.806312,2.297040
1209,1209.0,94119.048262,NaN,88398.586879,1298.024079,8905.109893,10.294486,103347.543915,1.087571e+06,106082.959731,2.345450,1.074165e+06,49595.639929,1057.051949,103147.267707,NaN,104.099172,11275.080883,115055.523086,105201.547958,10.358605,2.872759,107834.745605,10.070086,2708.039945,90.315251,4502.083820,2267.035496,206892.199058,1.016735e+06,8545.036877,NaN,818.975938,95.255808,2614.046035,1002.028549,11.262573,3.016052,10912.480045,1.016656e+06,...,10.447547,NaN,10.502198,967.481047,88.216016,10.535296,3548.968573,3.358649e+16,8140.800029,98.172030,11.651412,8779.078077,7.920442e+05,10.525088,10.468168,104071.834129,2.615953,130.309663,10513.170240,2.228708,728.052692,1.147362e+06,9.968195,1077.035231,7580.079750,8509.033486,10228.351982,10327.801413,100144.263391,310.183226,10.887052,10.836007,1124.755976,988.193910,102076.305244,101964.413583,6470.242459,9.786964,NaN,2.222794
1210,1210.0,NaN,3356.949591,97081.843123,1002.778571,10864.284014,10.797471,102894.681201,NaN,100102.393811,2.436208,1.072574e+06,37587.588730,1047.896435,105241.465707,93.329187,102.255382,11668.076593,115055.525882,85619.926316,10.732048,2.422995,104044.942484,NaN,2945.014802,112.182044,3202.074919,4883.134639,206892.208501,1.095387e+06,7036.096752,8.290676,878.350888,93.612258,2560.094596,1011.331590,10.580875,2.822595,NaN,1.085728e+06,...,10.115872,2.15810,9.868292,1004.134878,110.999990,10.713020,3394.243287,1.335970e+17,8371.821670,89.703650,9.220535,8998.055593,1.113826e+06,10.302250,10.883418,101283.417629,2.186525,366.750195,11376.166095,2.186734,1120.066965,1.013800e+06,11.734373,1046.939030,9132.087999,5896.007658,10706.113864,10369.803817,102404.821085,NaN,10.661966,10.569144,1010.143125,1064.316139,101477.589227,104517.364548,4922.920835,8.566389,103968.235402,2.071553
1211,1211.0,101744.439395,

In [0]:
column_names_y = ['id','y']
raw_dataset_y = pd.read_csv('/content/y_train.csv', names=column_names_y,
                      na_values = "?", comment='\t',
                      sep=",", skipinitialspace=True, skiprows=True)

dataset_y = raw_dataset_y.copy()
dataset_y.tail()

,id,y
1207,1207.0,66.0
1208,1208.0,73.0
1209,1209.0,74.0
1210,1210.0,78.0
1211,1211.0,64.0


# Print the missing values

In [0]:
missing_values = dataset_x.isna().sum()
print (missing_values)

id        0
x0       81
x1      103
x2       92
x3       91
       ... 
x827     83
x828     78
x829     98
x830     84
x831     92
Length: 833, dtype: int64


# Split the data into training and test data

In [0]:
# Split using sklearn.model_selection
x_train, x_test, y_train, y_test = train_test_split(dataset_x, dataset_y, test_size=0.2, random_state = 100)

In [0]:
train_stats = x_train.describe()
train_stats.pop("id")
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
x0,909.0,99849.359545,9534.020258,65533.368423,93818.485147,100183.062423,105994.290528,130226.576502
x1,885.0,3698.730375,943.683864,180.312021,3076.550570,3651.110055,4303.892503,7265.213902
x2,891.0,99975.389109,9540.065988,68544.573581,93937.346571,99386.035114,106102.200889,132221.045067
x3,898.0,999.944996,100.903669,694.745271,935.303439,999.571797,1068.606823,1434.200505
x4,890.0,10001.743350,1001.473353,6681.561828,9339.312428,10021.924636,10646.003276,13560.223285
...,...,...,...,...,...,...,...,...
x827,906.0,104990.967566,2761.209013,100015.768596,102783.100024,104986.305216,107352.370223,109999.847537
x828,907.0,6827.704539,1387.835714,1696.036569,6002.316474,6835.947954,7652.607118,11276.075121
x829,884.0,10.021817,0.982265,6.899008,9.378562,9.977236,10.676450,13.188278
x830,902.0,104960.353706,2845.423469,100003.049706,102653.373914,104838.184005,107428.898901,109993.046071


# Fill the NaN in the training data set with the median values of each column

In [0]:
x_train = x_train.fillna(x_train.median())
x_test = x_test.fillna(x_test.median())
missing_values = x_train.isna().sum()
print (missing_values)

id      0
x0      0
x1      0
x2      0
x3      0
       ..
x827    0
x828    0
x829    0
x830    0
x831    0
Length: 833, dtype: int64


# Remove the unnecessary "id" label

In [0]:
y_train.pop("id")
y_test.pop("id")

1143    1143.0
941      941.0
365      365.0
467      467.0
615      615.0
         ...  
156      156.0
689      689.0
28        28.0
69        69.0
1198    1198.0
Name: id, Length: 243, dtype: float64

# Feature Extraction using autofeat

In [0]:
fsel = FeatureSelector(featsel_runs=4,
        max_it=150,
        w_thr=1e-6,
        keep=None,
        n_jobs=1,
        verbose=1)

new_X = fsel.fit_transform(pd.DataFrame(x_train, columns=column_names_x), y_train)
print(new_X.columns) 

df_train = pd.DataFrame(x_train, columns=column_names_x)
df_test = pd.DataFrame(x_test, columns=column_names_x)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[featsel] Scaling data...done.
[featsel] 220/833 features after univariate filtering
[featsel] Feature selection run 1/4
[featsel] Feature selection run 2/4
[featsel] Feature selection run 3/4
[featsel] Feature selection run 4/4
[featsel] 28 features after 4 feature selection runs
[featsel] 28 features after correlation filtering
[featsel] 21 features after noise filtering
[featsel] 21 final features selected (including 0 original keep features).
Index(['x400', 'x635', 'x757', 'x516', 'x809', 'x214', 'x556', 'x617', 'x93',
       'x346', 'x596', 'x255', 'x309', 'x252', 'x292', 'x738', 'x537', 'x593',
       'x474', 'x614', 'x502'],
      dtype='object')


# Keep only the necessary features

In [0]:
dataset_selected_x = x_train.copy()
x_test_selected= x_test.copy()

#accepted=[400, 757, 635, 516, 132, 15, 809, 116, 214,
       #556, 617, 93, 346, 596, 309, 252, 292, 474,
       #593, 614]

accepted=[400, 635, 757, 516, 809, 214, 556, 617, 93,
       346, 596, 255, 309, 252, 292, 738, 537, 593,
       474, 614, 502]

for j in range(832):
        if (j in accepted):
          print (j)
        else:
          del dataset_selected_x['x'+str(j)]
          del x_test_selected['x'+str(j)]
      
train_stats = dataset_selected_x.describe()
train_stats.pop("id")
train_stats = train_stats.transpose()
train_stats

93
214
252
255
292
309
346
400
474
502
516
537
556
593
596
614
617
635
738
757
809


,count,mean,std,min,25%,50%,75%,max
x93,969.0,3.519955e+03,6.123839e+02,1.789851e+03,3.122307e+03,3.492040e+03,3.882096e+03,5.902057e+03
x214,969.0,1.686814e+03,4.246669e+02,3.857756e+02,1.438003e+03,1.657997e+03,1.929010e+03,3.719025e+03
x252,969.0,5.153136e+03,6.443209e+03,-1.066389e+04,2.036529e+03,3.051817e+03,5.396992e+03,5.218476e+04
x255,969.0,1.063172e+04,1.847941e+03,3.258064e+03,9.580092e+03,1.053304e+04,1.170004e+04,1.742768e+04
x292,969.0,1.289525e+05,1.561997e+04,7.115248e+04,1.201669e+05,1.282218e+05,1.377145e+05,1.982293e+05
x309,969.0,1.360934e+04,2.104294e+03,1.787475e+03,1.241801e+04,1.355888e+04,1.480791e+04,2.142239e+04
x346,969.0,7.264234e+03,1.238998e+03,2.195927e+03,6.577637e+03,7.355575e+03,8.085946e+03,1.121581e+04
x400,969.0,2.419177e+00,1.566607e-01,1.441218e+00,2.325897e+00,2.416256e+00,2.507563e+00,3.029658e+00
x474,969.0,2.138704e+05,3.363320e+04,6.580233e+04,1.951336e+05,2.113702e+05,2.311579e+05,4.824998e+05
x502,969.0,7.341196e+13,5.051213e+13,-8.384285e+13,4.129257e+13,6.245895e+13,9.198935e+13,3.816907e+14


# Regression model

In [0]:
rfr = ExtraTreesRegressor(n_jobs=1, max_depth=None, n_estimators=180, random_state=0, min_samples_split=3, max_features=None)

rfr.fit(dataset_selected_x, np.ravel(y_train))

y_pred = rfr.predict(x_test_selected) 

score = r2_score(y_test, y_pred)
print(score)

0.6037280104368132


# Export the file with our predictions for submission

In [0]:
#accepted=[400, 757, 635, 516, 132, 15, 809, 116, 214,
       #556, 617, 93, 346, 596, 309, 252, 292, 474,
       #593, 614]

accepted=[400, 635, 757, 516, 809, 214, 556, 617, 93,
       346, 596, 255, 309, 252, 292, 738, 537, 593,
       474, 614, 502]

column_names_x = ['id']
for i in range(832):
  column_names_x.append('x'+str(i))

raw_dataset_x_test = pd.read_csv('/content/X_test.csv', names=column_names_x,
                      na_values = "?", comment='\t',
                      sep=",", skipinitialspace=True, skiprows=True)

dataset_x_test = raw_dataset_x_test.copy()
dataset_x_test.tail()

dataset_x_test = dataset_x_test.fillna(dataset_x_test.median())

dataset_selected_x_test = dataset_x_test.copy()

for j in range(832):
        if (j in accepted):
          print (j)
        else:
          del dataset_selected_x_test['x'+str(j)]

predictions = rfr.predict(dataset_selected_x_test)

index = 0.0
with open('predictions.txt', 'w') as f:
    f.write("%s\n" % "id,y")
    for predict in predictions:
        writing_str = str(index)+','+str(predict.item(0))
        f.write("%s\n" % writing_str)
        index = index + 1

93
214
252
255
292
309
346
400
474
502
516
537
556
593
596
614
617
635
738
757
809
